In [1]:
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv('20100008.csv')
df = df[df["Adjustments"] == "Unadjusted"]

canada_df = df[df["GEO"] == "Canada"]

canada_df_few_cols = canada_df[["North American Industry Classification System (NAICS)", "REF_DATE", "VALUE"]]
canada_df_pivot = canada_df_few_cols.pivot(index=canada_df_few_cols["REF_DATE"], columns='North American Industry Classification System (NAICS)')['VALUE']

# All stores that do not have data at the start
canada_df_pivot_fewer_cols = canada_df_pivot.drop(columns=["Automobile dealers [4411]",
                            "Automotive parts, accessories and tire stores [4413]", 
                            "Cannabis stores [453993]", "Clothing stores [4481]", 
                            "Clothing stores [4481]", "Convenience stores [44512]", 
                            "Grocery stores [4451]", "Jewellery, luggage and leather goods stores [4483]", 
                            "Other motor vehicle dealers [4412]", "Shoe stores [4482]", 
                            "Specialty food stores [4452]", "Used car dealers [44112]"])

# All classifications that do not have data at the end
canada_df_pivot_fewer_cols = canada_df_pivot_fewer_cols.drop(columns=["Department stores [4521]", "Other general merchandise stores [4529]"])
canada_df_pivot_nona = canada_df_pivot_fewer_cols.dropna()

# Looking at Beer store sales vs everthing else
Are there some that are strongly correlated or not?

In [2]:
X = canada_df_pivot_nona[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]', 'Furniture and home furnishings stores [442]',
                         'Furniture stores [4421]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Home furnishings stores [4422]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'New car dealers [44111]', 'Retail trade [44-45]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona["Beer, wine and liquor stores [4453]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                                     OLS Regression Results                                    
Dep. Variable:     Beer, wine and liquor stores [4453]   R-squared:                       0.981
Model:                                             OLS   Adj. R-squared:                  0.980
Method:                                  Least Squares   F-statistic:                     1147.
Date:                                 Wed, 30 Dec 2020   Prob (F-statistic):          2.82e-281
Time:                                         15:31:49   Log-Likelihood:                -4497.8
No. Observations:                                  357   AIC:                             9028.
Df Residuals:                                      341   BIC:                             9090.
Df Model:                                           15                                         
Covariance Type:                             nonrobust                                         
                                        

In [3]:
X = canada_df_pivot_nona[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Food and beverage stores [445]', 'Gasoline stations [447]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona["Beer, wine and liquor stores [4453]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                                     OLS Regression Results                                    
Dep. Variable:     Beer, wine and liquor stores [4453]   R-squared:                       0.970
Model:                                             OLS   Adj. R-squared:                  0.970
Method:                                  Least Squares   F-statistic:                     2284.
Date:                                 Wed, 30 Dec 2020   Prob (F-statistic):          3.26e-265
Time:                                         15:31:49   Log-Likelihood:                -4574.2
No. Observations:                                  357   AIC:                             9160.
Df Residuals:                                      351   BIC:                             9184.
Df Model:                                            5                                         
Covariance Type:                             nonrobust                                         
                                        

# Dividing all values by total retail trade
This will get rid of the upward trend that exists because inflation is a thing.

In [4]:
# calling it normalized because I don't know what else to call it
canada_df_pivot_nona_normalized = pd.DataFrame()
for (columnName, columnData) in canada_df_pivot_nona.iteritems():
    canada_df_pivot_nona_normalized[columnName] = canada_df_pivot_nona[columnName] / canada_df_pivot_nona["Retail trade [44-45]"]

In [5]:
X = canada_df_pivot_nona_normalized[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]', 'Furniture and home furnishings stores [442]',
                         'Furniture stores [4421]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Home furnishings stores [4422]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'New car dealers [44111]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona_normalized["Beer, wine and liquor stores [4453]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                                     OLS Regression Results                                    
Dep. Variable:     Beer, wine and liquor stores [4453]   R-squared:                       0.676
Model:                                             OLS   Adj. R-squared:                  0.663
Method:                                  Least Squares   F-statistic:                     50.99
Date:                                 Wed, 30 Dec 2020   Prob (F-statistic):           7.22e-75
Time:                                         15:31:49   Log-Likelihood:                 1594.7
No. Observations:                                  357   AIC:                            -3159.
Df Residuals:                                      342   BIC:                            -3101.
Df Model:                                           14                                         
Covariance Type:                             nonrobust                                         
                                        

In [6]:
X = canada_df_pivot_nona_normalized[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'New car dealers [44111]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona_normalized["Beer, wine and liquor stores [4453]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                                     OLS Regression Results                                    
Dep. Variable:     Beer, wine and liquor stores [4453]   R-squared:                       0.625
Model:                                             OLS   Adj. R-squared:                  0.614
Method:                                  Least Squares   F-statistic:                     52.38
Date:                                 Wed, 30 Dec 2020   Prob (F-statistic):           7.84e-67
Time:                                         15:31:49   Log-Likelihood:                 1568.8
No. Observations:                                  357   AIC:                            -3114.
Df Residuals:                                      345   BIC:                            -3067.
Df Model:                                           11                                         
Covariance Type:                             nonrobust                                         
                                        

In [7]:
X = canada_df_pivot_nona_normalized[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Food and beverage stores [445]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'New car dealers [44111]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona_normalized["Beer, wine and liquor stores [4453]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

#
# The new car sales are interesting
#

                                     OLS Regression Results                                    
Dep. Variable:     Beer, wine and liquor stores [4453]   R-squared:                       0.623
Model:                                             OLS   Adj. R-squared:                  0.612
Method:                                  Least Squares   F-statistic:                     57.10
Date:                                 Wed, 30 Dec 2020   Prob (F-statistic):           3.55e-67
Time:                                         15:31:49   Log-Likelihood:                 1567.5
No. Observations:                                  357   AIC:                            -3113.
Df Residuals:                                      346   BIC:                            -3070.
Df Model:                                           10                                         
Covariance Type:                             nonrobust                                         
                                        

In [9]:
est2_robust = est2.get_robustcov_results()

print(est2_robust.summary())

                                     OLS Regression Results                                    
Dep. Variable:     Beer, wine and liquor stores [4453]   R-squared:                       0.623
Model:                                             OLS   Adj. R-squared:                  0.612
Method:                                  Least Squares   F-statistic:                     61.12
Date:                                 Wed, 30 Dec 2020   Prob (F-statistic):           2.35e-70
Time:                                         15:32:43   Log-Likelihood:                 1567.5
No. Observations:                                  357   AIC:                            -3113.
Df Residuals:                                      346   BIC:                            -3070.
Df Model:                                           10                                         
Covariance Type:                                   HC1                                         
                                        

# Look at new car dealers now

In [10]:
X = canada_df_pivot_nona[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]', 'Furniture and home furnishings stores [442]',
                         'Furniture stores [4421]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Home furnishings stores [4422]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'Beer, wine and liquor stores [4453]', 'Retail trade [44-45]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona["New car dealers [44111]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                               OLS Regression Results                              
Dep. Variable:     New car dealers [44111]   R-squared:                       0.999
Model:                                 OLS   Adj. R-squared:                  0.999
Method:                      Least Squares   F-statistic:                 2.240e+04
Date:                     Wed, 30 Dec 2020   Prob (F-statistic):               0.00
Time:                             15:52:36   Log-Likelihood:                -4548.2
No. Observations:                      357   AIC:                             9128.
Df Residuals:                          341   BIC:                             9190.
Df Model:                               15                                         
Covariance Type:                 nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [11]:
X = canada_df_pivot_nona[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'Beer, wine and liquor stores [4453]', 'Retail trade [44-45]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona["New car dealers [44111]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                               OLS Regression Results                              
Dep. Variable:     New car dealers [44111]   R-squared:                       0.999
Model:                                 OLS   Adj. R-squared:                  0.999
Method:                      Least Squares   F-statistic:                 2.804e+04
Date:                     Wed, 30 Dec 2020   Prob (F-statistic):               0.00
Time:                             15:54:13   Log-Likelihood:                -4549.5
No. Observations:                      357   AIC:                             9125.
Df Residuals:                          344   BIC:                             9175.
Df Model:                               12                                         
Covariance Type:                 nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [12]:
X = canada_df_pivot_nona[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'Beer, wine and liquor stores [4453]', 'Retail trade [44-45]']]
y = canada_df_pivot_nona["New car dealers [44111]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                               OLS Regression Results                              
Dep. Variable:     New car dealers [44111]   R-squared:                       0.999
Model:                                 OLS   Adj. R-squared:                  0.999
Method:                      Least Squares   F-statistic:                 3.065e+04
Date:                     Wed, 30 Dec 2020   Prob (F-statistic):               0.00
Time:                             15:55:13   Log-Likelihood:                -4549.6
No. Observations:                      357   AIC:                             9123.
Df Residuals:                          345   BIC:                             9170.
Df Model:                               11                                         
Covariance Type:                 nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

## Now Do some tests on New Car Dealers When everything is divided by total retail trade

In [13]:
X = canada_df_pivot_nona_normalized[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]', 'Furniture and home furnishings stores [442]',
                         'Furniture stores [4421]', 'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Home furnishings stores [4422]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'Beer, wine and liquor stores [4453]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona_normalized["New car dealers [44111]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                               OLS Regression Results                              
Dep. Variable:     New car dealers [44111]   R-squared:                       0.992
Model:                                 OLS   Adj. R-squared:                  0.992
Method:                      Least Squares   F-statistic:                     3192.
Date:                     Wed, 30 Dec 2020   Prob (F-statistic):               0.00
Time:                             15:58:02   Log-Likelihood:                 1652.5
No. Observations:                      357   AIC:                            -3275.
Df Residuals:                          342   BIC:                            -3217.
Df Model:                               14                                         
Covariance Type:                 nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [14]:
X = canada_df_pivot_nona_normalized[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Electronics and appliance stores [443]', 'Food and beverage stores [445]',
                         'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]', 'Miscellaneous store retailers [453]',
                         'Motor vehicle and parts dealers [441]', 'Beer, wine and liquor stores [4453]',
                         'Sporting goods, hobby, book and music stores [451]']]
y = canada_df_pivot_nona_normalized["New car dealers [44111]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                               OLS Regression Results                              
Dep. Variable:     New car dealers [44111]   R-squared:                       0.992
Model:                                 OLS   Adj. R-squared:                  0.992
Method:                      Least Squares   F-statistic:                     4067.
Date:                     Wed, 30 Dec 2020   Prob (F-statistic):               0.00
Time:                             15:59:22   Log-Likelihood:                 1651.2
No. Observations:                      357   AIC:                            -3278.
Df Residuals:                          345   BIC:                            -3232.
Df Model:                               11                                         
Covariance Type:                 nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [17]:
X = canada_df_pivot_nona_normalized[['Building material and garden equipment and supplies dealers [444]', 'Clothing and clothing accessories stores [448]',
                         'Food and beverage stores [445]',
                         'Gasoline stations [447]', 'General merchandise stores [452]', 
                         'Health and personal care stores [446]',
                         'Motor vehicle and parts dealers [441]', 'Beer, wine and liquor stores [4453]']]
y = canada_df_pivot_nona_normalized["New car dealers [44111]"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                               OLS Regression Results                              
Dep. Variable:     New car dealers [44111]   R-squared:                       0.992
Model:                                 OLS   Adj. R-squared:                  0.992
Method:                      Least Squares   F-statistic:                     5560.
Date:                     Wed, 30 Dec 2020   Prob (F-statistic):               0.00
Time:                             16:02:52   Log-Likelihood:                 1648.6
No. Observations:                      357   AIC:                            -3279.
Df Residuals:                          348   BIC:                            -3244.
Df Model:                                8                                         
Covariance Type:                 nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [18]:
est2_robust = est2.get_robustcov_results()

print(est2_robust.summary())

                               OLS Regression Results                              
Dep. Variable:     New car dealers [44111]   R-squared:                       0.992
Model:                                 OLS   Adj. R-squared:                  0.992
Method:                      Least Squares   F-statistic:                     5155.
Date:                     Wed, 30 Dec 2020   Prob (F-statistic):               0.00
Time:                             16:02:58   Log-Likelihood:                 1648.6
No. Observations:                      357   AIC:                            -3279.
Df Residuals:                          348   BIC:                            -3244.
Df Model:                                8                                         
Covariance Type:                       HC1                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------